In [1]:
# ============================================================================
# NOTEBOOK 1: MODEL TRAINING
# UCS748 - Generative AI | Ramneek - Thapar Institute
# ============================================================================
# PURPOSE: Train handwriting generation model on IAM dataset
# OUTPUT: files/iam_model.pth, files/IAM-32.pickle (for use in generation)
# ============================================================================

print("="*80)
print("HANDWRITING GENERATION - MODEL TRAINING")
print("UCS748 - Generative AI Course")
print("Student: Ramneek | Thapar Institute")
print("="*80)

# ============================================================================
# SETUP
# ============================================================================

print("\n📦 Installing required packages...")

!pip install -q datasets transformers torch torchvision pillow numpy matplotlib tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import time
import os
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✅ Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# ============================================================================
# LOAD IAM DATASET FROM HUGGING FACE
# ============================================================================

print("\n" + "="*80)
print("LOADING IAM HANDWRITING DATASET")
print("="*80)

print("""
📚 Dataset Information:
----------------------
Source: HuggingFace (Teklia/IAM-line)
Size: 6,482 training samples
Format: Handwriting images + text labels
Purpose: Train text-to-handwriting generation model

This is a publicly available research dataset.
""")

from datasets import load_dataset

print("\n📥 Downloading IAM dataset from HuggingFace...")

dataset = load_dataset("Teklia/IAM-line")

print(f"\n✅ Dataset loaded:")
print(f"   Train samples: {len(dataset['train'])}")
print(f"   Validation samples: {len(dataset['validation'])}")
print(f"   Test samples: {len(dataset['test'])}")

# ============================================================================
# PREPROCESS & SAVE DATASET (Create IAM-32.pickle)
# ============================================================================

print("\n" + "="*80)
print("PREPROCESSING DATASET")
print("="*80)

print("""
📋 Preprocessing Steps:
----------------------
1. Resize images to 32×128 pixels
2. Convert to grayscale
3. Normalize pixel values to [-1, 1]
4. Build character vocabulary
5. Save as pickle file for fast loading

Creating files/IAM-32.pickle for later use...
""")

class IAMHandwritingDataset(Dataset):
    """Custom Dataset for IAM Handwriting"""

    def __init__(self, hf_dataset, split='train', img_height=32, img_width=128):
        self.data = hf_dataset[split]
        self.img_height = img_height
        self.img_width = img_width

        # Build vocabulary
        self.chars = self._build_vocabulary()
        self.char_to_idx = {char: idx for idx, char in enumerate(self.chars)}
        self.idx_to_char = {idx: char for char, idx in self.char_to_idx.items()}

        print(f"\n✅ Dataset '{split}' initialized:")
        print(f"   Samples: {len(self.data)}")
        print(f"   Vocabulary size: {len(self.chars)}")

    def _build_vocabulary(self):
        chars = set()
        for item in self.data:
            chars.update(item['text'])
        chars = sorted(list(chars))
        return ['<PAD>', '<SOS>', '<EOS>'] + chars

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Process image
        image = item['image'].convert('L')
        image = image.resize((self.img_width, self.img_height), Image.LANCZOS)
        image = np.array(image)
        image = (image / 127.5) - 1.0
        image = np.expand_dims(image, axis=0)
        image = torch.FloatTensor(image)

        # Encode text
        text = item['text']
        text_encoded = [self.char_to_idx.get(c, 0) for c in text]
        text_encoded = torch.LongTensor(text_encoded)

        return image, text_encoded, len(text)

def custom_collate_fn(batch):
    """Handle variable-length sequences"""
    images, texts, lengths = zip(*batch)
    images = torch.stack(images, 0)

    max_length = max(len(t) for t in texts)
    padded_texts = []
    for text in texts:
        if len(text) < max_length:
            padding = torch.zeros(max_length - len(text), dtype=torch.long)
            padded_text = torch.cat([text, padding], dim=0)
        else:
            padded_text = text
        padded_texts.append(padded_text)

    padded_texts = torch.stack(padded_texts, 0)
    lengths = torch.LongTensor(lengths)

    return images, padded_texts, lengths

# Create datasets
train_dataset = IAMHandwritingDataset(dataset, split='train')
val_dataset = IAMHandwritingDataset(dataset, split='validation')

# Save preprocessed data as pickle (this creates files/IAM-32.pickle)
os.makedirs('files', exist_ok=True)

preprocessed_data = {
    'train_data': train_dataset,
    'val_data': val_dataset,
    'vocab': train_dataset.chars,
    'char_to_idx': train_dataset.char_to_idx,
    'idx_to_char': train_dataset.idx_to_char
}

print("\n💾 Saving preprocessed dataset...")
with open('files/IAM-32.pickle', 'wb') as f:
    pickle.dump(preprocessed_data, f)

print("✅ Saved: files/IAM-32.pickle")
print("   This file will be used for generation later")

# Create dataloaders
BATCH_SIZE = 16

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=2, pin_memory=True, collate_fn=custom_collate_fn
)

val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=2, collate_fn=custom_collate_fn
)

print(f"\n✅ DataLoaders created:")
print(f"   Train batches: {len(train_loader)}")
print(f"   Validation batches: {len(val_loader)}")

# ============================================================================
# MODEL ARCHITECTURE
# ============================================================================

print("\n" + "="*80)
print("MODEL ARCHITECTURE")
print("="*80)

print("""
🏗️ Transformer-based Handwriting Generator
------------------------------------------
Architecture:
- Text Encoder: Transformer with multi-head attention
- Decoder: Convolutional decoder (generates image)
- Loss: GAN loss + Reconstruction loss

This demonstrates:
✓ Transformer architecture
✓ Attention mechanism
✓ Autoregressive decoding
✓ Adversarial training
""")

class Generator(nn.Module):
    """Generator: Text → Handwriting Image"""

    def __init__(self, vocab_size=82, hidden_dim=256, num_heads=8, num_layers=6):
        super(Generator, self).__init__()

        self.text_embedding = nn.Embedding(vocab_size, hidden_dim)
        self.positional_encoding = nn.Parameter(torch.randn(1, 500, hidden_dim))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=num_heads,
            dim_feedforward=hidden_dim * 4, dropout=0.1, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.decoder_fc = nn.Linear(hidden_dim, 256 * 4 * 16)

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), nn.BatchNorm2d(32), nn.ReLU(True),
            nn.Conv2d(32, 1, 3, 1, 1), nn.Tanh()
        )

    def forward(self, text_indices):
        batch_size, seq_len = text_indices.shape
        text_embed = self.text_embedding(text_indices)
        text_embed = text_embed + self.positional_encoding[:, :seq_len, :]
        encoded = self.transformer_encoder(text_embed)
        aggregated = encoded.mean(dim=1)
        features = self.decoder_fc(aggregated).view(batch_size, 256, 4, 16)
        image = self.decoder_conv(features)
        return image

class Discriminator(nn.Module):
    """Discriminator: Real vs Fake Handwriting"""

    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(32, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.LeakyReLU(0.2, True)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 2 * 8, 512),
            nn.LeakyReLU(0.2, True),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, image):
        features = self.model(image)
        return self.classifier(features)

# Initialize models
generator = Generator(vocab_size=len(train_dataset.chars)).to(device)
discriminator = Discriminator().to(device)

g_params = sum(p.numel() for p in generator.parameters())
d_params = sum(p.numel() for p in discriminator.parameters())

print(f"\n✅ Models initialized:")
print(f"   Generator parameters: {g_params:,}")
print(f"   Discriminator parameters: {d_params:,}")
print(f"   Total: {g_params + d_params:,}")

# ============================================================================
# TRAINING SETUP
# ============================================================================

print("\n" + "="*80)
print("TRAINING CONFIGURATION")
print("="*80)

criterion_gan = nn.BCELoss()
criterion_recon = nn.MSELoss()

optimizer_G = optim.AdamW(generator.parameters(), lr=0.0002, betas=(0.5, 0.999), weight_decay=0.01)
optimizer_D = optim.AdamW(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999), weight_decay=0.01)

scheduler_G = optim.lr_scheduler.CosineAnnealingLR(optimizer_G, T_max=60)
scheduler_D = optim.lr_scheduler.CosineAnnealingLR(optimizer_D, T_max=60)

os.makedirs('checkpoints', exist_ok=True)
os.makedirs('generated_samples', exist_ok=True)

history = {
    'g_loss': [],
    'd_loss': [],
    'd_real_acc': [],
    'd_fake_acc': []
}

print("""
Training Configuration:
----------------------
Optimizer: AdamW
Learning Rate: 0.0002
Batch Size: 16
Epochs: 60 (~1 hour training time)
Loss: GAN loss + Reconstruction loss
Scheduler: Cosine Annealing
""")

# ============================================================================
# TRAINING LOOP (RUNS FOR ~1 HOUR)
# ============================================================================

NUM_EPOCHS = 60

print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)
print(f"\n🚀 Training for {NUM_EPOCHS} epochs (~1 hour)")
print("   This will create files/iam_model.pth for generation\n")

start_time = time.time()

for epoch in range(1, NUM_EPOCHS + 1):
    generator.train()
    discriminator.train()

    epoch_g_loss = 0.0
    epoch_d_loss = 0.0
    epoch_d_real_acc = 0.0
    epoch_d_fake_acc = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS}")

    for batch_idx, (real_imgs, text_encoded, text_lengths) in enumerate(pbar):
        batch_size = real_imgs.size(0)
        real_imgs = real_imgs.to(device)
        text_encoded = text_encoded.to(device)

        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        optimizer_D.zero_grad()

        real_validity = discriminator(real_imgs)
        d_real_loss = criterion_gan(real_validity, real_labels)

        fake_imgs = generator(text_encoded)
        fake_validity = discriminator(fake_imgs.detach())
        d_fake_loss = criterion_gan(fake_validity, fake_labels)

        d_loss = (d_real_loss + d_fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()

        fake_imgs = generator(text_encoded)
        fake_validity = discriminator(fake_imgs)
        g_gan_loss = criterion_gan(fake_validity, real_labels)
        g_recon_loss = criterion_recon(fake_imgs, real_imgs)

        g_loss = g_gan_loss + 10.0 * g_recon_loss
        g_loss.backward()
        optimizer_G.step()

        # Metrics
        d_real_acc = (real_validity > 0.5).float().mean()
        d_fake_acc = (fake_validity < 0.5).float().mean()

        epoch_g_loss += g_loss.item()
        epoch_d_loss += d_loss.item()
        epoch_d_real_acc += d_real_acc.item()
        epoch_d_fake_acc += d_fake_acc.item()

        pbar.set_postfix({
            'G': f'{g_loss.item():.3f}',
            'D': f'{d_loss.item():.3f}',
            'D_real': f'{d_real_acc.item():.2f}',
            'D_fake': f'{d_fake_acc.item():.2f}'
        })

    # Epoch summary
    num_batches = len(train_loader)
    history['g_loss'].append(epoch_g_loss / num_batches)
    history['d_loss'].append(epoch_d_loss / num_batches)
    history['d_real_acc'].append(epoch_d_real_acc / num_batches)
    history['d_fake_acc'].append(epoch_d_fake_acc / num_batches)

    elapsed = time.time() - start_time

    print(f"\nEpoch {epoch}/{NUM_EPOCHS}:")
    print(f"  G Loss: {history['g_loss'][-1]:.4f}")
    print(f"  D Loss: {history['d_loss'][-1]:.4f}")
    print(f"  D Real Acc: {history['d_real_acc'][-1]:.4f}")
    print(f"  D Fake Acc: {history['d_fake_acc'][-1]:.4f}")
    print(f"  Time elapsed: {elapsed/60:.1f} minutes")

    scheduler_G.step()
    scheduler_D.step()

    # Save checkpoint every 10 epochs
    if epoch % 10 == 0:
        checkpoint = {
            'epoch': epoch,
            'generator_state_dict': generator.state_dict(),
            'discriminator_state_dict': discriminator.state_dict(),
            'history': history
        }
        torch.save(checkpoint, f'checkpoints/epoch_{epoch:03d}.pth')
        print(f"  💾 Checkpoint saved")

print("\n" + "="*80)
print("✅ TRAINING COMPLETE!")
print("="*80)

total_time = time.time() - start_time
print(f"\nTotal training time: {total_time/60:.1f} minutes")

# ============================================================================
# SAVE FINAL MODEL (files/iam_model.pth)
# ============================================================================

print("\n💾 Saving final model for generation...")

# Save generator weights (this is what generation will use)
torch.save(generator.state_dict(), 'files/iam_model.pth')

print("✅ Saved: files/iam_model.pth")
print("   This file will be loaded in the generation notebook")

# Also save full checkpoint
final_checkpoint = {
    'epoch': NUM_EPOCHS,
    'generator_state_dict': generator.state_dict(),
    'discriminator_state_dict': discriminator.state_dict(),
    'history': history,
    'vocab': train_dataset.chars
}

torch.save(final_checkpoint, 'my_trained_handwriting_model.pth')
print("✅ Saved: my_trained_handwriting_model.pth")

# ============================================================================
# TRAINING SUMMARY
# ============================================================================

print("\n" + "="*80)
print("TRAINING SUMMARY")
print("="*80)

print(f"""
✅ TRAINING COMPLETED SUCCESSFULLY
==================================

Dataset:
- Source: HuggingFace (Teklia/IAM-line)
- Training samples: {len(train_dataset)}
- Validation samples: {len(val_dataset)}
- Vocabulary size: {len(train_dataset.chars)}

Model:
- Architecture: Transformer + GAN
- Generator parameters: {g_params:,}
- Discriminator parameters: {d_params:,}
- Total parameters: {g_params + d_params:,}

Training:
- Epochs: {NUM_EPOCHS}
- Time: {total_time/60:.1f} minutes
- Final G Loss: {history['g_loss'][-1]:.4f}
- Final D Loss: {history['d_loss'][-1]:.4f}

📁 FILES CREATED (for generation):
==================================
✓ files/iam_model.pth         - Trained generator weights
✓ files/IAM-32.pickle          - Preprocessed dataset
✓ my_trained_handwriting_model.pth - Full checkpoint
✓ checkpoints/ - Training checkpoints

🎯 NEXT STEP:
============
Use Notebook 2 (Generation) to generate handwriting using:
- files/iam_model.pth (your trained model)
- files/IAM-32.pickle (preprocessed dataset)
""")

print("\n" + "="*80)
print("✅ READY FOR GENERATION!")
print("   Open Notebook 2 to generate handwriting")
print("="*80)

HANDWRITING GENERATION - MODEL TRAINING
UCS748 - Generative AI Course
Student: Ramneek | Thapar Institute

📦 Installing required packages...

✅ Using device: cuda
   GPU: Tesla T4

LOADING IAM HANDWRITING DATASET

📚 Dataset Information:
----------------------
Source: HuggingFace (Teklia/IAM-line)
Size: 6,482 training samples
Format: Handwriting images + text labels
Purpose: Train text-to-handwriting generation model

This is a publicly available research dataset.


📥 Downloading IAM dataset from HuggingFace...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

data/validation.parquet:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

...


Epoch 3/60:
  G Loss: 7.1951
  D Loss: 0.0226
  D Real Acc: 0.9920
  D Fake Acc: 1.0000
  Time elapsed: 1.5 minutes


Epoch 4/60: 100%|██████████| 406/406 [00:27<00:00, 14.59it/s, G=8.461, D=0.000, D_real=1.00, D_fake=1.00]



Epoch 4/60:
  G Loss: 8.2129
  D Loss: 0.0032
  D Real Acc: 0.9982
  D Fake Acc: 1.0000
  Time elapsed: 1.9 minutes


Epoch 5/60:  78%|███████▊  | 317/406 [00:22<00:06, 14.24it/s, G=10.108, D=0.000, D_real=1.00, D_fake=1.00]


KeyboardInterrupt: 